In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import os
import numpy as np

In [2]:
from shapely.geometry import Point, Polygon

In [3]:
sns.set(style="darkgrid")

In [4]:
filepath = os.path.join("Resources", "data.csv")
data = pd.read_csv(filepath)
data.head()

,neighbourhood
0,Waterfront Communities-The Island
1,Newtonbrook East
2,York University Heights
3,Dorset Park
4,Englemount-lawrence


In [5]:
nb = os.path.join('Resources', 'Neighbourhoods', "Neighbourhoods.shp")
regions = gpd.read_file(nb)
regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()
regions.sample(5)

,FIELD_1,FIELD_2,FIELD_3,FIELD_4,FIELD_5,FIELD_6,FIELD_7,FIELD_8,FIELD_9,FIELD_10,FIELD_11,FIELD_12,FIELD_13,FIELD_14,FIELD_15,geometry,neighbourhood
94,2195,25886378,25926756,49885,109,109,Caledonia-Fairbank (109),Caledonia-Fairbank (109),None,None,-79.455212,43.688569,16493009,2.955857e+06,6849.911724,"POLYGON ((-79.46021 43.68156, -79.46044 43.681...",caledonia-fairbank
76,2177,25886794,25926738,49885,20,20,Alderwood (20),Alderwood (20),None,None,-79.541611,43.604937,16492721,9.502180e+06,12667.013917,"POLYGON ((-79.54866 43.59022, -79.54876 43.590...",alderwood
30,2131,25886548,25926692,49885,121,121,Oakridge (121),Oakridge (121),None,None,-79.279708,43.697408,16491985,3.596139e+06,10807.516883,"POLYGON ((-79.26986 43.69584, -79.27009 43.695...",oakridge
92,2193,25886711,25926754,49885,30,30,Brookhaven-Amesbury (30),Brookhaven-Amesbury (30),None,None,-79.485589,43.701326,16492977,6.715562e+06,12417.055559,"POLYGON ((-79.50296 43.69574, -79.50415 43.696...",brookhaven-amesbury
31,2132,25886374,25926693,49885,107,107,Oakwood Village (107),Oakwood Village (107),None,None,-79.439785,43.688566,16492001,4.247608e+06,8766.961761,"POLYGON ((-79.45028 43.69569, -79.44934 43.695...",oakwood village


In [6]:
regions.to_csv('Resources/regions.csv')

In [7]:
starbucks_df = pd.read_csv('Resources/Starbucks locations.csv')
starbucks_df.head()

,ID,Lat,Long
0,75921-104040,43.086574,-79.059356
1,3997-146205,43.077276,-79.082792
2,75525-35359,43.078906,-79.081879
3,75633-86381,43.079703,-79.082204
4,75790-96396,43.082189,-79.082469


In [8]:
# Create a set of starbucks locations lat and lng combinations
lng_lats = []

lngs = starbucks_df['Long']
lats = starbucks_df['Lat']

lng_lats = zip(lngs, lats)

neighbourhood_id = []
neighbourhood_name = []
lats_list = []
lngs_list = []

for lng_lat in lng_lats:
    point = Point(lng_lat[0], lng_lat[1])
    
    for i in np.arange(len(regions)):
        poly = regions.loc[i, 'geometry']
        
        if point.within(poly):
            neighbourhood_id.append(regions.loc[i, 'FIELD_6'])
            neighbourhood_name.append(regions.loc[i, 'neighbourhood'])
            lats_list.append(lng_lat[1])
            lngs_list.append(lng_lat[0])

In [9]:
starbucks_dict = {
    'Lat' : lats_list,
    'Long' : lngs_list,
    'Hood ID' : neighbourhood_id,
    'Neighbourhood' : neighbourhood_name
}

clean_starbucks = pd.DataFrame(starbucks_dict)

clean_starbucks

,Lat,Long,Hood ID,Neighbourhood
0,43.797471,-79.148805,131,rouge
1,43.743753,-79.216334,139,scarborough village
2,43.770374,-79.186485,136,west hill
3,43.816306,-79.293317,130,milliken
4,43.776324,-79.259292,127,bendale
...,...,...,...,...
173,43.766139,-79.476335,27,york university heights
174,43.726773,-79.451830,31,yorkdale-glen park
175,43.757568,-79.465829,27,york university heights
176,43.725807,-79.450222,31,yorkdale-glen park


In [10]:
# number of stores for each location
starbucks_val_counts = clean_starbucks['Neighbourhood'].value_counts()
starbucks_val_counts = starbucks_val_counts.reset_index()
starbucks_val_counts.columns = ['Neighbourhood', 'Number of Stores']

starbucks_val_counts

,Neighbourhood,Number of Stores
0,bay street corridor,26
1,waterfront communities-the island,19
2,church-yonge corridor,8
3,islington-city centre west,8
4,annex,7
...,...,...
67,yonge-eglinton,1
68,rouge,1
69,kingsway south,1
70,scarborough village,1


In [11]:
clean_starbucks_df = clean_starbucks.drop_duplicates(subset='Neighbourhood', keep='first')
clean_starbucks_df

,Lat,Long,Hood ID,Neighbourhood
0,43.797471,-79.148805,131,rouge
1,43.743753,-79.216334,139,scarborough village
2,43.770374,-79.186485,136,west hill
3,43.816306,-79.293317,130,milliken
4,43.776324,-79.259292,127,bendale
...,...,...,...,...
164,43.698454,-79.463205,108,briar hill-belgravia
167,43.665399,-79.471892,90,junction area
169,43.651357,-79.475875,88,high park north
170,43.725315,-79.454165,31,yorkdale-glen park


In [12]:
starbucks_final = pd.merge(starbucks_val_counts, clean_starbucks_df, on='Neighbourhood', how='inner')
starbucks_final.drop(['Lat', 'Long'], axis='columns', inplace=True)

In [13]:
starbucks_final = starbucks_final[['Hood ID','Neighbourhood','Number of Stores']]
starbucks_final.rename(columns={"Hood ID": "Hood_ID"}, inplace=True)

In [14]:
starbucks_final

,Hood_ID,Neighbourhood,Number of Stores
0,76,bay street corridor,26
1,77,waterfront communities-the island,19
2,75,church-yonge corridor,8
3,14,islington-city centre west,8
4,95,annex,7
...,...,...,...
67,100,yonge-eglinton,1
68,131,rouge,1
69,15,kingsway south,1
70,139,scarborough village,1


In [15]:
starbucks_final.to_csv('output_data/starbucks_final.csv')